### Imports

In [2]:
import pandas as pd
import glob
import datetime
import os, shutil

### Path set-up

In [3]:
if 'DATA_DIR' not in locals():
    DATA_DIR = './data/'
else:
    print(DATA_DIR)

if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(os.path.dirname(DATA_DIR), exist_ok=True)

In [4]:
if 'OUTPUT_DATA_FOLDER' not in locals():
    OUTPUT_DATA_FOLDER = './output/'
else:
    print(OUTPUT_DATA_FOLDER)

if os.path.exists(OUTPUT_DATA_FOLDER) and os.path.isdir(OUTPUT_DATA_FOLDER):
    shutil.rmtree(OUTPUT_DATA_FOLDER)
os.makedirs(os.path.dirname(OUTPUT_DATA_FOLDER), exist_ok=True)

In [5]:
if 'ELASTIC_INDEX' not in locals():
    ELASTIC_INDEX = 'siren'
else:
    print(ELASTIC_INDEX)

### Import data

In [6]:
# Import Stock Unite Legale data
df_unite_legale = pd.read_csv('https://files.data.gouv.fr/insee-sirene/StockUniteLegale_utf8.zip', compression='zip', dtype=str, usecols=['siren', 
       'dateCreationUniteLegale', 'sigleUniteLegale',
       'prenom1UniteLegale','identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale', 
       'dateDernierTraitementUniteLegale', 'categorieEntreprise','etatAdministratifUniteLegale',
       'nomUniteLegale', 'denominationUniteLegale', 'categorieJuridiqueUniteLegale',
       'activitePrincipaleUniteLegale', 'economieSocialeSolidaireUniteLegale'])

In [7]:
# Rename columns
df_unite_legale = df_unite_legale.rename(columns={
    'dateCreationUniteLegale': 'date_creation_entreprise', 
    'sigleUniteLegale': 'sigle',
    'prenom1UniteLegale': 'prenom',
    'trancheEffectifsUniteLegale': 'tranche_effectif_salarie_entreprise',
    'dateDernierTraitementUniteLegale': 'date_mise_a_jour',
    'categorieEntreprise': 'categorie_entreprise',
    'nomUniteLegale': 'nom',
    'denominationUniteLegale': 'nom_raison_sociale',
    'categorieJuridiqueUniteLegale': 'nature_juridique_entreprise',
    'activitePrincipaleUniteLegale': 'activite_principale_entreprise'
})

### Data preprocessing

In [8]:
 def nom_complet(x):
    if(x['nature_juridique_entreprise'] == '1000'):
        if(x['sigle'] == x['sigle']):
            if((x['prenom'] == x['prenom']) & (x['nom'] == x['nom'])):
                return x['prenom'].lower()+' '+x['nom'].lower()+' ('+x['sigle'].lower()+')'
            else:
                return None
        else:
            if((x['prenom'] == x['prenom']) & (x['nom'] == x['nom'])):
                return x['prenom'].lower()+' '+x['nom'].lower()
            else: 
                return None
    else:
        if(x['sigle'] == x['sigle']):
            if(x['nom_raison_sociale'] == x['nom_raison_sociale']):
                return x['nom_raison_sociale'].lower()+' ('+x['sigle'].lower()+')'
            else:
                return None
        else:
            if(x['nom_raison_sociale'] == x['nom_raison_sociale']):
                return x['nom_raison_sociale'].lower()
            else:
                return None

In [9]:
# Add nom_complet column to df_unite_legale
df_unite_legale['nom_complet'] = df_unite_legale.apply(lambda row: nom_complet(row), axis=1)

KeyboardInterrupt: 

In [ ]:
# Create list of departement zip codes
all_deps = [*'-0'.join(list(str(x) for x in range(0, 10))).split('-')[1:],
    *list(str(x) for x in range(10,20)),
    *['2A','2B'],
    *list(str(x) for x in range(21,95)),
    *'-7510'.join(list(str(x) for x in range(0, 10))).split('-')[1:],
    *'-751'.join(list(str(x) for x in range(10, 21))).split('-')[1:],
    *['']
    ]
# Remove Paris zip code
all_deps.remove('75')

In [ ]:
# Upload geo data by departement
for dep in all_deps:
    url = 'https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_'+dep+'.csv.gz'
    print(url)
    df_dep = pd.read_csv(
            url,
            compression="gzip",
            dtype=str,
            usecols=['siren', 'siret',
           'dateCreationEtablissement', 'trancheEffectifsEtablissement',
           'activitePrincipaleRegistreMetiersEtablissement',
           'etablissementSiege',
           'numeroVoieEtablissement',
           'libelleVoieEtablissement',
           'codePostalEtablissement', 'libelleCommuneEtablissement',
            'codeCommuneEtablissement',
            'dateDebut', 'etatAdministratifEtablissement', 'enseigne1Etablissement',
           'activitePrincipaleEtablissement',
           'geo_adresse', 'longitude', 'latitude', 'indiceRepetitionEtablissement']
        )
    df_dep = df_dep.rename(columns={
            'dateCreationEtablissement': 'date_creation',
            'trancheEffectifsEtablissement': 'tranche_effectif_salarie',
            'activitePrincipaleRegistreMetiersEtablissement': 'activite_principale_registre_metier',
            'etablissementSiege': 'is_siege',
            'numeroVoieEtablissement': 'numero_voie',
            'typeVoieEtablissement': 'type_voie',
            'libelleVoieEtablissement': 'libelle_voie',
            'codePostalEtablissement': 'code_postal',
            'libelleCommuneEtablissement': 'libelle_commune',
            'codeCommuneEtablissement': 'commune',
            'codeCedexEtablissement': 'cedex',
            'dateDebut_x': 'date_debut_activite', 
            'etatAdministratifEtablissement': 'etat_administratif_etablissement',
            'enseigne1Etablissement': 'enseigne',
            'activitePrincipaleEtablissement': 'activite_principale',
            'indiceRepetitionEtablissement': 'indice_repetition'
        
        })
    df_dep.to_csv(DATA_DIR+'geo_siret_'+dep+'.csv',index=False)

In [ ]:
# Get geo data file paths
geo_files = glob.glob(DATA_DIR+"geo_siret*.csv")

In [ ]:
geo_files.sort()

In [ ]:
# Compute nbre d'établissements' per 'siren'
df_out=pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_geo['file'] = geo_file
    df_geo['nombre_etablissements'] = df_geo.groupby(['siren','file'])['siret'].transform('count')
    df_enseigne = df_geo.groupby(['siren','file'])['enseigne'].apply(set).reset_index(name='liste_enseigne').drop(columns=['file'], axis=1)
    df_geo = df_geo.merge(df_enseigne, left_on='siren', right_on='siren')
    df_adresse = df_geo.groupby(['siren','file'])['geo_adresse'].apply(set).reset_index(name='liste_adresse').drop(columns=['file'], axis=1)
    df_geo = df_geo.merge(df_adresse, left_on='siren', right_on='siren')
    df_inter = df_geo[['siren','file','nombre_etablissements', 'liste_enseigne', 'liste_adresse']]
    df_out = pd.concat([df_out, df_inter])
df_out = df_out.drop_duplicates(subset='siren',keep='first')
df_out2 = df_out[['siren','nombre_etablissements']].groupby(['siren'],as_index=False).sum()
df_out2 = df_out2.merge(df_out[['liste_enseigne', 'liste_adresse','siren']], on='siren', how='left')
df_unite_legale = pd.merge(df_unite_legale,df_out2,on='siren',how='left')

In [ ]:
# Compute 'nombre etablissements ouvert' per 'siren'
df_out=pd.DataFrame()
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_geo = df_geo[df_geo['etat_administratif_etablissement'] == 'A']
    df_geo['file'] = geo_file
    df_geo['nombre_etablissements_ouvert'] = df_geo.groupby(['siren','file'])['siret'].transform('count')
    df_inter = df_geo[['siren','file','nombre_etablissements_ouvert']]
    df_out = pd.concat([df_out, df_inter])
df_out = df_out.drop_duplicates(keep='first')
df_out2 = df_out[['siren','nombre_etablissements_ouvert']].groupby(['siren'],as_index=False).sum()
df_unite_legale = pd.merge(df_unite_legale,df_out2,on='siren',how='left')

In [ ]:
# Merge geo files with above dataframe
for geo_file in geo_files:
    print(geo_file)
    df_geo = pd.read_csv(geo_file,dtype=str)
    df_inter = pd.merge(df_geo,df_unite_legale,on='siren',how='left')
    df_inter2 = df_inter[df_inter['is_siege'] == 'true']
    df_inter2['concat_nom_adr_siren'] = df_inter2['nom_complet'] + ' ' + df_inter2['geo_adresse'] + ' ' + df_inter2['siren']
    df_inter2['concat_enseigne_adresse'] = df_inter2.apply(lambda x: x.liste_enseigne.union(x.liste_adresse), axis=1)
    df_inter.to_csv(OUTPUT_DATA_FOLDER+'siret_'+geo_file.replace(DATA_DIR+'geo_siret_',''), index=False)
    df_inter2.to_csv(OUTPUT_DATA_FOLDER+ELASTIC_INDEX+'_'+geo_file.replace(DATA_DIR+'geo_siret_',''), index=False)